In [223]:
import pandas as pd
import sqlite3

# Нормализация БД относительно актеров

## Установка подключения

In [224]:
connection = sqlite3.connect('/content/netflix.sqlite')

## Инициализация итератора по результатам запросов

In [225]:
cursor = connection.cursor()

## Чтение таблицы в DataFrame films

In [226]:
films = pd.read_sql("SELECT * FROM netflix_titles", connection)
films.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,81145628,Movie,Norm of the North: King Sized Adventure,"Richard Finn, Tim Maltby","Alan Marriott, Andrew Toth, Brian Dobson, Cole...","United States, India, South Korea, China","September 9, 2019",2019,TV-PG,90 min,"Children & Family Movies, Comedies",Before planning an awesome wedding for his gra...
1,80117401,Movie,Jandino: Whatever it Takes,,Jandino Asporaat,United Kingdom,"September 9, 2016",2016,TV-MA,94 min,Stand-Up Comedy,Jandino Asporaat riffs on the challenges of ra...
2,70234439,TV Show,Transformers Prime,,"Peter Cullen, Sumalee Montano, Frank Welker, J...",United States,"September 8, 2018",2013,TV-Y7-FV,1 Season,Kids' TV,"With the help of three human allies, the Autob..."


## Создание таблицы actors

### Вспомогательная функция для выборки актеров

In [227]:
def split_actors(string):
  return string.split(", ")

### Выборка + Удаление повторяющихся имен актеров и ненужных значений

In [228]:
cast = [split_actors(string) for string in films['cast'].unique()]
resampled_cast = []
for obj in cast:
  resampled_cast += [str(actor) for actor in obj]
cast = list(set(resampled_cast))[1:]
cast.sort()
cast[0:10]

[' Jr.',
 '2 Chainz',
 '4Minute',
 '50 Cent',
 'A Boogie Wit tha Hoodie',
 'A-ra Go',
 'A. Murat Özgen',
 'A.C. Peterson',
 'A.D. Miles',
 'A.J. Cook']

### Создание DataFrame actors

In [229]:
actor_id = [int(i) for i in range(len(cast))]

actors = pd.DataFrame({
    "actor_id": actor_id,
    "actor_name": cast
})
actors

,actor_id,actor_name
0,0,Jr.
1,1,2 Chainz
2,2,4Minute
3,3,50 Cent
4,4,A Boogie Wit tha Hoodie
...,...,...
27400,27400,Şenay Gürler
27401,27401,Şerif Sezer
27402,27402,Şinasi Yurtsever
27403,27403,Şükran Ovalı


### Создание таблицы actors в базе данных

In [230]:
with connection:
  cursor.execute("""
  CREATE TABLE actors (
    actor_id INT,
    actor_name VARCHAR(50),
    PRIMARY KEY(actor_id)
    );
  """)
  connection.commit()

### Запись данных DataFrame actors в таблицу actors

In [231]:
actors.to_sql("actors", con=connection, if_exists="append", index=False)
connection.commit()

In [232]:
cursor.execute("SELECT * FROM actors").fetchmany(10)

[(0, ' Jr.'),
 (1, '2 Chainz'),
 (2, '4Minute'),
 (3, '50 Cent'),
 (4, 'A Boogie Wit tha Hoodie'),
 (5, 'A-ra Go'),
 (6, 'A. Murat Özgen'),
 (7, 'A.C. Peterson'),
 (8, 'A.D. Miles'),
 (9, 'A.J. Cook')]

## Создание таблицы actors_films

### Выборка нужного набора фильмов для каждого актера

In [233]:
actors_films_dataframes = []
for actor in cast:
  curr_film_set = films[films['cast'].apply(lambda x: actor in x.split(", "))]['show_id']
  curr_actor_set = [cast.index(actor)] * len(curr_film_set.to_list())
  actors_films_dataframes.append(pd.DataFrame({
      'actor_id': curr_actor_set,
      'show_id': curr_film_set.to_list()
  }))
actors_films_dataframes


[   actor_id   show_id
 0         0  80025601,
    actor_id   show_id
 0         1  80145087,
    actor_id   show_id
 0         2  80161826,
    actor_id   show_id
 0         3  70098605
 1         3  70199239
 2         3  80013712,
    actor_id   show_id
 0         4  80145087,
    actor_id   show_id
 0         5  80214013,
    actor_id   show_id
 0         6  81043541,
    actor_id   show_id
 0         7  81090899,
    actor_id   show_id
 0         8  60021299
 1         8  80039813
 2         8  80117800,
    actor_id   show_id
 0         9  60026114
 1         9  70153390,
    actor_id   show_id
 0        10  80141858
 1        10  80049870
 2        10  80075595,
    actor_id   show_id
 0        11  17969262
 1        11  70065769
 2        11  60003405
 3        11  81221914,
    actor_id   show_id
 0        12  80216270,
    actor_id   show_id
 0        13  81063743,
    actor_id   show_id
 0        14  80174974,
    actor_id   show_id
 0        15  80106440,
    actor_id   sho

### Преобразование полученной выборки в DataFrame путём слияния

In [234]:
actors_films = pd.concat(actors_films_dataframes)
actors_films['actor_id'] = actors_films['actor_id'].astype (int)
actors_films['show_id'] = actors_films['show_id'].astype (int)
actors_films

,actor_id,show_id
0,0,80025601
0,1,80145087
0,2,80161826
0,3,70098605
1,3,70199239
...,...,...
1,27402,80134500
2,27402,80227119
0,27403,80227113
0,27404,80119381


### Создание таблицы actors_films в базе данных

In [235]:
with connection:
  cursor.execute("""
  CREATE TABLE actors_films (
    actor_id INT,
    show_id INT,
    PRIMARY KEY(actor_id, show_id),
    FOREIGN KEY(actor_id) REFERENCES actors(actor_id),
    FOREIGN KEY(show_id) REFERENCES netflix_titles(show_id)
    );
  """)
  connection.commit()

### Запись данных DataFrame actors_films в таблицу actors_films

In [236]:
actors_films.to_sql("actors_films", con=connection, if_exists="append", index=False)
connection.commit()

In [237]:
cursor.execute("SELECT * FROM actors_films").fetchmany(10)

[(0, 80025601),
 (1, 80145087),
 (2, 80161826),
 (3, 70098605),
 (3, 70199239),
 (3, 80013712),
 (4, 80145087),
 (5, 80214013),
 (6, 81043541),
 (7, 81090899)]

In [242]:
connection.close()

# Наиболее часто работаящая друг с другом пара актеров

## Откроем новую базу данных, полученную из прошлого задания

In [249]:
connection = sqlite3.connect('/content/task1.sqlite')
cursor = connection.cursor()

## Запрос на выборку id актеров пары наиболее работающих друг с другом актеров

In [250]:
pare = cursor.execute(
  """
  SELECT af1.actor_id, af2.actor_id, COUNT(*) as frequency
  FROM actors_films af1
  JOIN actors_films af2
      ON af1.show_id = af2.show_id
      AND af1.actor_id != af2.actor_id
  GROUP BY af1.actor_id, af2.actor_id
  ORDER BY frequency DESC
  LIMIT 1;
  """
).fetchone()[1::]
pare

(21274, 15)

## Запрос на выборку пары наиболее работающих друг с другом актеров

In [251]:
pare = cursor.execute(
    """
    SELECT actor_name FROM actors
    WHERE actor_id = 21274 OR actor_id = 15;
    """
).fetchmany(2)
pare

[('AJ Bowen',), ('Robb Wells',)]

In [252]:
connection.close()

# ***Ответ***

In [253]:
answer = pare[0][0] + " and " + pare[1][0]
answer

'AJ Bowen and Robb Wells'